## 5. Analisis de Sentimiento con HuggingFace

### 5.1 Instalacion de Dependencias

In [1]:
!pip install transformers
!pip install torch torchvision torchaudio

### 5.2 Importaciones

In [2]:
import pandas as pd
import numpy as np
from transformers import pipeline
import torch
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score
from sklearn.metrics import classification_report, confusion_matrix

import warnings
warnings.filterwarnings('ignore')

2025-12-07 17:52:35.350153: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


### 5.3 Carga de Datos

In [3]:
#Carga dataset
df = pd.read_pickle('Outputs/data/df_beauty_balanced.pkl')

print(f"Dataset cargado: {len(df):,} reviews")
df.head()

Dataset cargado: 6,000 reviews


,review,rating
0,Sculpting Crean Use this product and find that...,5.0
1,Keep your money Foe the price one expects more...,1.0
2,Fell apart after a year Was good while it last...,1.0
3,Five Stars Works beautifully. Great for my cli...,5.0
4,Worst Product I recently purchased this produc...,1.0


In [4]:
review_col = "review"

### 5.3 Cargar Modelo

Utilizamos DistilBERT, como modelo transformer pre-entrenado para analisis de sentimiento.

In [6]:
# Cargar modelo de HuggingFace
# Forzamos el uso de PyTorch (framework='pt') para evitar conflictos con TensorFlow/Keras
sentiment_model = pipeline(
    "sentiment-analysis",
    model="distilbert-base-uncased-finetuned-sst-2-english",
    framework="pt",  # Forzar PyTorch
    device=-1  # -1 = CPU, 0 = GPU
)

print("Modelo DistilBERT cargado exitosamente")
print("Framework: PyTorch")
print("Dispositivo: CPU")

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use cpu


Modelo DistilBERT cargado exitosamente
Framework: PyTorch
Dispositivo: CPU


### 5.4 Prueba con Ejemplos

Probamos el modelo con algunas reviews del dataset.

In [7]:
# Probar con algunas reviews del dataset
sample_reviews = df[review_col].head(5).tolist()

print("Ejemplos de predicciones:\n")
for i, review in enumerate(sample_reviews, 1):
    # Truncar review para visualizacion
    review_short = review[:100] + "..." if len(review) > 100 else review
    
    # Predecir sentimiento
    result = sentiment_model(review, truncation=True, max_length=512)[0]
    
    print(f"{i}. Review: {review_short}")
    print(f"   Prediccion: {result['label']} (confianza: {result['score']:.4f})")
    print(f"   Rating real: {df.iloc[i-1]['rating']}\n")

Ejemplos de predicciones:

1. Review: Sculpting Crean Use this product and find that when I run out, I notice the difference in the tautne...
   Prediccion: POSITIVE (confianza: 0.9794)
   Rating real: 5.0

2. Review: Keep your money Foe the price one expects more than eye shadow....you could get what you need from t...
   Prediccion: NEGATIVE (confianza: 0.9990)
   Rating real: 1.0

3. Review: Fell apart after a year Was good while it lasted but that wasn't long. Brush fell apart. Don't waste...
   Prediccion: NEGATIVE (confianza: 0.9984)
   Rating real: 1.0

4. Review: Five Stars Works beautifully. Great for my clients with sensitive skin.
   Prediccion: POSITIVE (confianza: 0.9999)
   Rating real: 5.0

5. Review: Worst Product I recently purchased this product and it was a terrible experience!! I used this glue ...
   Prediccion: NEGATIVE (confianza: 0.9998)
   Rating real: 1.0



### 5.5 Predicciones en el Dataset Completo

In [ ]:
from tqdm import tqdm
import numpy as np

# Preparar datos
reviews = df[review_col].tolist()

# Crear etiquetas binarias (1=positivo, 0=negativo)
df['sentiment'] = (df['rating'] >= 3).astype(int)

print(f"Total de reviews a procesar: {len(reviews):,}")

# Realizar predicciones en lotes
batch_size = 32
predictions = []
confidences = []

for i in tqdm(range(0, len(reviews), batch_size), desc="Procesando"):
    batch = reviews[i:i+batch_size]
    
    # Predecir en lote
    results = sentiment_model(batch, truncation=True, max_length=512)
    
    for result in results:
        pred = 1 if result['label'] == 'POSITIVE' else 0
        predictions.append(pred)
        confidences.append(result['score'])

# Añadir predicciones al dataframe
df['prediction'] = predictions
df['confidence'] = confidences

print(f"\nPredicciones completadas: {len(predictions):,} reviews")

Total de reviews a procesar: 6,000


Procesando: 100%|██████████| 188/188 [01:57<00:00,  1.60it/s]


Predicciones completadas: 6,000 reviews


### 5.6 Evaluacion del Modelo

In [9]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Calcular metricas
y_true = df['sentiment']
y_pred = df['prediction']
y_conf = df['confidence']

accuracy = accuracy_score(y_true, y_pred)
precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted')
roc_auc = roc_auc_score(y_true, y_conf)


print("RESULTADOS: DistilBERT (HuggingFace)")
print(f"Accuracy:  {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1-Score:  {f1:.4f}")
print(f"ROC-AUC:   {roc_auc:.4f}")

# Reporte detallado
print("\nReporte de Clasificacion:\n")
print(classification_report(y_true, y_pred, target_names=['Negativo', 'Positivo']))

RESULTADOS: DistilBERT (HuggingFace)
Accuracy:  0.8787
Precision: 0.8835
Recall:    0.8787
F1-Score:  0.8783
ROC-AUC:   0.5830

Reporte de Clasificacion:

              precision    recall  f1-score   support

    Negativo       0.84      0.94      0.89      3000
    Positivo       0.93      0.82      0.87      3000

    accuracy                           0.88      6000
   macro avg       0.88      0.88      0.88      6000
weighted avg       0.88      0.88      0.88      6000



### 5.9 Conclusiones

- Modelo transformer pre-entrenado que ofrece conocimiento transferido de grandes corpus de texto, no requiere feature engineering manual (TF-IDF, etc.)
- Funciona bien con texto completo sin preprocesamiento
- Tiempo de inferencia mas lento (varios minutos para 6K reviews)
- Pre-entrenado en textos generales, no en reviews de productos especificamente


#### Conclusion General

DistilBERT demuestra el poder de los modelos pre-entrenados de HuggingFace para analisis de sentimiento. Aunque requiere mas recursos que los modelos tradicionales, ofrece una alternativa solida cuando se necesita precision y se cuenta con la infraestructura adecuada.